In [ ]:
## UNCOMMENT TO INSTALL BEFORE RUNNING FIRST TIME
#!pip install selenium
#!pip install webdriver-manager

import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import time 
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

import numpy as np
import pandas as pd
import re

In [ ]:
def scrape_attraction(link):
    
    driver.get(link)
    time.sleep(1)
    
    info = []
    
    # setting some of the parameters with default values
    att_site = None
    att_phone = None
    att_add = None
    top_choice = 0
    
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[2]/main')))
    
    # finding the name and city of the attraction
    att_name = driver.find_element(by=By.XPATH, value='//*[@id="__next"]/div[2]/main/div[1]/header/div[1]/h1').text
    att_city = driver.find_element(by=By.XPATH, value='//*[@id="__next"]/div[2]/main/div[1]/header/div[1]/div[2]/p').text
    # cleanup and top choice check
    if ("Top choice" in att_city):
        top_choice = 1
        att_city = re.sub('Top choice', '', att_city)
        att_city = re.sub('in ', '', att_city)
    att_city = att_city.strip()
    if (att_city == ''):
        att_city = None
    
    try: 
        img_link = driver.find_element(by=By.XPATH, value='//*[@id="__next"]/div[2]/main/section/div/img').get_attribute("src")
    except NoSuchElementException: 
        img_link = None
        
    try: 
        desc = driver.find_element(by=By.XPATH, value='//*[@id="__next"]/div[2]/main/div[1]/div/article[1]/section/div').text
    except NoSuchElementException: 
        desc = None
    
    # locating the info block and extracting available info
    block = driver.find_element(by=By.XPATH, value='//*[@id="__next"]/div[2]/main/div[1]/div/article[2]/div[1]').text
    for text in block.split("\n"):
        if (text == "ADDRESS") or (text == "GET IN TOUCH") or (text == "CONTACT"):
            continue
        if "." in text:
            att_site = text
        else:
            if (re.search('[a-zA-Z]', text) == None):
                att_phone = text
            else:
                att_add = text

    info.append(att_name)
    info.append(att_city)
    info.append(top_choice)
    info.append(att_add)
    info.append(att_phone)
    info.append(att_site)
    info.append(desc)
    info.append(img_link)

    return info

In [ ]:
def clickNextPage():

    nextpage = driver.find_elements(by=By.XPATH, value ='//*[@id="__next"]/div[2]/main/div/section/div/div[1]/div[2]/a')
    action = webdriver.common.action_chains.ActionChains(driver)
    action.move_to_element_with_offset(nextpage[0], 10,10)
    action.click()
    action.perform()
    time.sleep(0.15)

In [ ]:
# GET ALL THE LINKS OF ALL THE ATTRACTIONS

# setting up selenium
ops = webdriver.ChromeOptions()
# uncomment to not see the pop-up site
#ops.headless = True
driver = webdriver.Chrome(options = ops)
#att_driver = webdriver.Chrome()
driver.get("https://www.lonelyplanet.com/israel/attractions")

# all categories links
cat_list = []

link_el = driver.find_elements(by=By.XPATH, value='//*[@id="__next"]/div[2]/main/div/section/nav/ul/li')
for li in link_el:
    if (li.text == "Hotels"):
        continue
    cat_list.append((li.find_element(by = By.TAG_NAME, value = "a").get_attribute("href"), li.text))

link_list = []

# running on all category links to get all attractions
cat_num = len(cat_list)
for i in range(cat_num):
    driver.get(cat_list[i][0])
    time.sleep(1)
    try: 
        nav_el = driver.find_element(by=By.XPATH, value='//*[@id="__next"]/div[2]/main/div/section/div/div[1]/ul').text
        pages = nav_el[-1]
    except NoSuchElementException: 
        pages = 1

    pages = int(pages)
    for j in range(pages):
        att_el = driver.find_elements(by=By.CLASS_NAME, value='col-span-1')
        for li in att_el:
            link_list.append((li.find_element(by = By.CLASS_NAME, value = "card-link").get_attribute("href"), cat_list[i][1]))
        if (pages != 1):
            clickNextPage()

In [ ]:
# setting up selenium
ops = webdriver.ChromeOptions()
# uncomment to not see the pop-up site
#ops.headless = True
driver = webdriver.Chrome(options = ops)

att_df = pd.DataFrame(columns = ['Attraction Category', 'Attraction Name' , 'Attraction City', 'Top Choice?' , 'Attraction Address',
                                'Attraction Phone', 'Attraction Site', 'Attraction Description', 'Image Link'])
att_num = len(link_list)
for att in range(att_num):
    try:
        inf_list = scrape_attraction(link_list[att][0])
        inf_list.insert(0, link_list[att][1])
        att_df.loc[len(att_df)] = inf_list
    except TimeoutException:
        driver.quit()
        time.sleep(20)
        driver = webdriver.Chrome(options = ops)
        att = att-1
driver.quit()

In [ ]:
att_df.to_csv('LonelyPlanetAttractions.csv', encoding="utf-8-sig")